# MidiNet: A Convolutional Generative Adversarial Network for Symbolic-domain Music Generation using 1D and 2D Conditions


#### Li-Chia Yang, Szu-Yu Chou, and Yi-Hsuan Yang
#### Research Center for Information Technology Innovation
#### Academia Sinica Taiwan∗

## Abstract

Neste artigo, apresentamos MidiNet, uma rede neural convoluta profunda (CNN)
Baseada na rede adversária generativa (GAN) que se destina a fornecer uma visão geral,
Estrutura de rede altamente adaptável para geração de música de domínio simbólico. o
A rede leva o ruído aleatório como entrada e gera uma seqüência de melodia uma medida
(Bar) após o outro. Além disso, tem um novo sub-modelo CNN reflexivo que
Nos permite orientar o processo de geração fornecendo não só 1D, mas também 2D
Condições. Em nossa implementação, usamos o acorde pretendido da barra atual
Como uma condição 1D para fornecer um contexto harmônico, e a melodia gerada para o
Antes da barra anteriormente como uma condição 2D para fornecer informações seqüenciais. o
A saída da rede é uma matriz de 16 por 128 cada vez, representando a presença de
Cada uma das 128 notas MIDI na sequência de melodia gerada dessa barra, com a
A menor unidade temporal é a décima sexta nota. MidiNet pode gerar música de
Número arbitrário de barras, concatenando estas 16 por 128 matrizes. A melodia
A sequência pode então ser reproduzida com um sintetizador. Nós fornecemos exemplos de clipes
Mostrando a eficácia da MidiNet na geração de música harmônica.

## Introduction

As redes adversárias generativas (GANs) [1, 2] ganharam uma tremenda atenção nos últimos anos.
Em um GAN, um modelo de gerador e um modelo discriminador são treinados simultaneamente sob o
Conceito de minimax de dois jogadores de teoria dos jogos. O modelo do gerador visa produzir dados artificiais
Isso poderia enganar o modelo discriminador, enquanto a tarefa do modelo discriminador é distinguir
Entre dados autênticos e artificiais. Para a diversidade do resultado da geração, a entrada para o
O modelo do gerador geralmente é um ruído aleatório. O modelo do gerador em um GAN bem-sucedido pode converter
Um ruído aleatório dado em algo que parece ser realista para o ser humano. Pode ser uma imagem, texto
Passagem ou clipe de som, dependendo dos dados de treinamento. Até à data, a maioria dos trabalhos existentes tem
Focado na geração de imagens usando GANs [3-5].



Para modelar a natureza temporal da música, uma série de trabalhos anteriores sobre geração de música são baseados em
Redes neurais recorrentes (RNNs) [6-10]. Embora essas RNNs às vezes possam produzir
Música, o treinamento de uma RNN geralmente é mais lento, em comparação com o de um neural convolucional
Rede (CNN) [11]. Também é menos direto adicionar restrições ou reguladores a uma RNN para informar o processo de geração com conhecimento musical a priori, que pode ser essencial para a qualidade estética do resultado da geração.

O objetivo deste artigo é propor um modelo GAN novo para geração de música de domínio simbólico que
É baseado na CNN, com foco na geração da sequência de melodia principal.2
Em vez de gerar um
Sequência de melodia continuamente, propomos gerar uma melodia uma medida (barra) após a outra,
De forma sucessiva. Isso nos permite usar uma matriz 2D que represente a ocorrência de diferentes
Notas ao longo do tempo como entrada para uma CNN. Além disso, aproveitando a idéia de contraditório condicional
Treinamento [5, 11, 12], propomos um novo sub-modelo CNN reflexivo que usa a sequência de melodia
Gerado anteriormente para a barra anterior (que é novamente representada como uma matriz 2D) para condicionar
A geração para a barra atual. Desta forma, a dependência temporal entre as melodias de
As barras sucessivas podem ser modeladas sem uma unidade recorrente, como é usado nas RNNs. A CNN reflexiva
Realmente nos permite usar as seqüências de melodia de várias barras anteriores para condicionar o GAN, para
Modelo de dependências temporais de longo prazo na música. Além de tais condições 2D, certamente podemos
Use as condições 1D mais simples (por exemplo, rótulos de classe), bem como para condicionar o GAN. Em outras palavras,
Dependendo da informação a priori que possamos, podemos adicionar diferentes componentes ao nosso GAN. este
Estrutura altamente adaptativa pode ser útil em vista dos padrões temporais complexos e hierárquicos (em
Tempo) e relações harmônicas (entre a melodia principal e os sons simultâneos e acompanhantes)
Encontrado na música. Nos referimos a este novo modelo CNN-GAN como MidiNet.

Observamos que, enquanto as condições de 1D foram amplamente utilizadas no trabalho anterior no GAN [5, 11, 12], a
O uso de uma condição 2D não foi tentado antes, conforme nosso conhecimento. O proposto
CNN reflexiva é necessária para adicionar essa condição 2D às camadas intermediárias de uma CNN, cada uma das
Que tem tamanhos diferentes. Embora usemos tais condições 2D para a aplicação específica de símbolos simbólicos
Geração de música de domínio, a idéia é realmente genérica e é aplicável a outros problemas
também. Ele abre nova porta para incluir informações laterais aos GANs. Mais detalhes do método são
Descrito na Seção 2.

Em nossa implementação (consulte a Seção 3), reunimos 1.022 guias MIDI para treinar a MidiNet. Nós costumavamos
A informação do acorde como a condição 1D, assumindo que a seqüência de acorde desejado é dada e
Que há apenas um acorde por barra. Além disso, a partir da segunda barra, usamos a melodia
Sequência gerada para a barra anterior como condição 2D. Em outras palavras, condicionamos a
Geração da melodia para cada barra por um rótulo de acorde e a melodia da barra anterior. Para
Estabilizar as atualizações de treinamento do GAN, usamos técnicas como a correspondência de recursos
E alinhamento de etiquetas unilateral [5]. MidiNet pode gerar música de número arbitrário de barras, por
Concatenando a sequência de melodia gerada para cada barra. Definimos o número alvo de barras para 8
E forneça exemplos do resultado da geração na Seção 4.

### 2.1 Generator model

Ao projetar CNNs para tarefas relacionadas à imagem, o uso de filtros convolucionais 2D para o aprendizado de recursos é comum [3,13]. Em contraste, para tarefas relacionadas ao áudio, às vezes é bom o suficiente para usar filtros convolucionais 1D, uma vez que os dados de áudio são seqüenciais [14, 15]. Portanto, usamos filtros da forma h-by-1 na última camada convolucional transposta, onde h indica o número de notas MIDI consideradas pela rede. Da mesma forma, a série de camadas convolucionais transpostas converte um vetor de valores aleatórios em uma matriz 2D da forma desejada h-by-w como a saída, o que corresponde à ocorrência de diferentes notas nas unidades temporais $w$. O tamanho dos filtros na primeira camada convolucional transposta é definido como 1-por-w. O modelo do gerador está ilustrado na Figura 1.

<img src="midinet.png"> 

### 2.2 Discriminator model

O modelo discriminador é uma CNN típica com duas camadas convolucionais e duas camadas densas. Isto é
Treinados para distinguir entre a representação 2D (ou seja, uma matriz h-por-w) de um real (isto é, autêntico)
Sequência de melodia de uma barra e a de uma sequência de melodia gerada (ou seja, artificial) de uma barra.

### 2.3 1D condition

Seguindo as ideias das redes adversárias generativas condicionais (CGANs) [12], adicionamos 1D
Não apenas as camadas de entrada do modelo discriminador e o modelo do gerador, mas também
Para todas as camadas intermediárias. Em geral, uma condição 1D é representada como um vetor n-dimensional.
Para adicioná-lo a uma camada intermediária de forma a-by-b, projetamos o vetor de condicionamento no mesmo
Forma da matriz para obter um tensor de forma a-by-b-by-n e concatenar com a camada intermediária em
O eixo do mapa de recursos. Isto é ilustrado pelos blocos laranja claro na Figura 1.

### 2.4 2D condition: reflective CNN

Como o resultado da geração do nosso GAN é uma matriz 2D de informações de tempo e freqüência, é plausível realizar o condicionamento em cada entrada da matriz 2D, levando a uma matriz condicional 2D.

Ao contrário do caso 1D, não é fácil remodelar uma matriz 2D em uma matriz de outro tamanho. Sem essa transformação, não seremos capazes de condicionar as camadas intermediárias dos modelos gerador e discriminador. A fim de mapear a condição 2D para as camadas intermediárias de uma maneira significativa, propomos formar outra CNN que tenha quase a mesma arquitetura (ou seja, número de camadas, número de filtros em cada camada e tamanho desses filtros) como o CNN do modelo gerador. No entanto, enquanto a CNN do modelo do gerador toma como entrada um vetor de números aleatórios e, como saída, a matriz $h$-by-$w$ desejada, a nova CNN leva exatamente a direção oposta - é preciso como entrada a matriz condicional $h$-by-$w$ e como Produza um vetor 1D.


Esta nova CNN é adicionada como uma sub-rede ao CNN do modelo do gerador e seus parâmetros (ou seja, pesos e termos tendenciosos dos filtros convolucionais) são aprendidos simultaneamente, conforme ilustrado pelos blocos azuis da Figura 1. Considerando a relação entre este novo CNN e a CNN do modelo gerador, nos referimos à nova CNN como CNN reflexiva neste artigo. 

<img src="tabela1.png"> 

Embora seja possível usar condições 2D para o modelo discriminador também, em nossa implementação optamos por não fazê-lo, por razões que serão esclarecidas na Seção 3.2.

Observamos que a saída de um GAN para geração de imagem também é uma matriz 2D, com cada entrada na matriz correspondente a um valor de pixel. Pode ser muito laborioso condicionar cada um desses pixels, embora também seja possível. Em contraste, é mais fácil criar uma matriz condicional 2D que tenha o mesmo tamanho que a saída da MidiNet, pois aqui a saída está em uma forma simbólica de notação de música. Esta é talvez uma das razões pelas quais as condições 2D não foram tentadas para a geração de imagens até agora.

## 3 Implementation

### 3.1 Dataset

Embora seja possível usar outros meios para condicionar a MidiNet, utilizamos na nossa implementação atual uma etiqueta de acordes para a condição 1D e a seqüência de melodia de uma barra anterior para a condição 2D, conforme descrito no último parágrafo da Seção 1. De acordo , Precisamos de uma coleção de guias Midi que incluam a melodia e as seqüências de acordes de múltiplas barras. Conseguimos isso rastejando a Internet, levando a uma coleção de 1.022 guias Midi com informações de acordes e melodias em canais Midi separados. Por simplicidade, filtramos todos os dados MIDI que contém acordes outros Do que as 24 tríades de acordes básicos (2 (maior e menor) * 12 (em uma oitava)). Em seguida, segmentamos as abas restantes pelo comprimento de 8 barras,e depois processou o canal de acordes e a melodia canal separadamente, conforme descrito abaixo.

Para os dados de melodia, nós reparamos a menor unidade de notas para a décima sexta nota. Especificamente, nós prolongamos as notas que têm uma nota de pausa após eles. Se a primeira nota de uma barra for uma nota de pausa, estendemos a segunda barra para que ela seja reproduzida enquanto a barra começa. Existem outras exceções de nota, como trigêmeos ou unidade de nota menor (por exemplo, 32ª anotações), mas optamos por excluir esses dados no treinamento da corrente
Versão da MidiNet. Consideramos todas as 128 notas MIDI (isto é, de C0 a G10). Consequentemente, uma sequência de melodia de uma barra pode ser representada como uma matriz de altura $h = 16$ e largura $w = 128$.

Para os dados de acordes, em vez de usar um vetor 24-dimensional one-hot para codificar os 24 tipos de acorde, achamos mais eficiente usar uma representação de acordes que tenha apenas 13 dimensões - a primeira dimensão para o tipo de acorde (ou seja, maior Ou menor) e os outros 12 para marcar as 3 notas na tríade, entre todas as 12 notas possíveis em uma oitava. Essas 12 dimensões de um acorde maior correspondem a 12 notas em uma oitava de acordo com a ordem, enquanto as de uma acordes menores correspondem ao menor relativo do acorde maior, conforme ilustrado na Tabela 1. Nós podamos os dados de acordes de modo que há apenas um acorde por barra.

Após o pré-processamento acima mencionado, ficamos com 526 fichas Midi, ou seja, 4,208 barras de pares de dados de melodia e acordes. Para ampliar o conjunto de treinamento, usamos transposição de chave para o aumento de dados. Ao transpor (ou seja, deslocando circularmente) a melodia e o acorde para qualquer uma das 12 chaves possíveis, nós finalmente tínhamos 50,496 (ou seja, 4,208 * 12) barras de pares de dados de melodia e acorde para treinar a MidiNet

### 3.2 Training of the MidiNet

Cada guia Midi em nosso conjunto de treinamento tem 8 barras, portanto, é possível condicionar a geração da seqüência de melodia de uma barra pelas seqüências de melodia das 7 barras anteriores, usando 7 matrizes condicionais. No entanto, isso só é possível para a última barra em cada guia Midi. Para fazer o melhor uso de todas as barras e também por simplicidade, consideramos apenas a barra anterior para a condição 2D em nossa implementação. Consequentemente, amostrámos 2 barras consecutivas cada vez que treinamos a MidiNet, usando a última para a melodia real e a anterior para a condição 2D.

Assumimos que o modelo discriminador deve ser capaz de distinguir uma melodia real de uma artificial sem conhecer a melodia de uma barra anterior. Portanto, utilizamos apenas a condição 1D para o modelo discriminador. Em contraste, utilizamos as condições 1D e 2D para o modelo do gerador. O uso de acorde como condição 1D fornece um contexto harmônico da melodia, tanto para os modelos discriminadores quanto para os geradores.

<img src="partitura_midinet.png"> 

Devido à natureza do equilíbrio de Nash de jogos não convexos, o treinamento de GANs está sujeito a problemas de instabilidade e colapso de modo [5]. Através de uma análise mais profunda do problema de otimização e da estrutura do modelo, algumas técnicas foram propostas recentemente para lidar com essas questões e, consequentemente, estabilizar as atualizações de treinamento de GANs [5,16,17]. Em nossa implementação, as técnicas de alinhamento de rótulos de correspondência e unilateral foram empregadas [5]. Quanto ao comprimento do vetor aleatório $z$ na experiência atual é definido como 100.

Observamos que, na fase de treinamento, o modelo do gerador só precisa gerar melodia de uma barra de cada vez, usando a melodia real da barra anterior para a condição 2D. No entanto, na fase de teste, o modelo do gerador deve gerar melodia de 8 barras sucessivamente, cada vez que usa a melodia (artificial) gerada anteriormente para a última barra como condição 2D.

## 4 Results

Nós demonstramos uma amostra gerada pela MidiNet na Figura 2. Na Figura 2 (a), apenas a condição do acorde 1D é usada, enquanto na Figura 2 (b), são utilizadas as condições 1D e 2D. Podemos ver que em ambos os casos MidiNet gera notas de melodia que também aparecem nos acordes. Isso mostra a eficácia da condição de acorde 1D. Além disso, vemos que a MidiNet pode gerar melodia com algumas notas repetidas quando a condição da barra anterior 2D é usada, o que melhora a conexão entre as barras adjacentes.

Para avaliar a qualidade estética e o prazer do resultado da geração, é necessário um estudo de escuta que envolva ouvintes humanos. Também pode ser formulado como um teste de Turing que pede ao ouvinte discriminar entre melodias reais e geradas. Embora o estudo de escuta ainda esteja em andamento, fornecemos exemplos de áudio do resultado de geração sintetizado on-line para uma avaliação subjetiva. Esses exemplos de áudio podem ser encontrados em https://richardyang40148.github.io/TheBlog/midinet_arxiv_demo.html.

Embora não seja relatado na versão atual do documento, o treinamento da MidiNet é eficiente, devido à estrutura da CNN. Nós também planejamos mostrar como a perda do discriminador e os modelos de geradores mudam em função das épocas de treinamento em uma versão posterior do documento.

## 5 Discussion and Conclusion

O design da MidiNet foi inspirado pela forma como as pessoas compuseram a música - primeiro temos uma sequência de acordes em mente e, em seguida, preenchemos a linha de melodia uma barra depois da outra.4 Também pretendemos manter a estrutura do MidiNet flexível, para que ela possa acomodar Outros conhecimentos e conhecimentos musicais adquiridos com a pesquisa de teoria musical. O que implementamos até agora considera apenas a relação entre melodia e acorde e a melodia entre barras adjacentes. Mover um O design da MidiNet foi inspirado pela forma como as pessoas compuseram a música - primeiro temos uma sequência de acordes em mente e, em seguida, preenchemos a linha de melodia uma barra depois da outra.4 Também pretendemos manter a estrutura do MidiNet flexível, para que ela possa acomodar Outros conhecimentos e conhecimentos musicais adquiridos com a pesquisa de teoria musical. O que implementamos até agora considera apenas a relação entre melodia e acorde e a melodia entre barras adjacentes. Avançando um passo adiante, planejamos considerar em nosso futuro trabalho uma hierarquia superior da música - as estruturas da música
Como a introdução, o verso e o coro (veja a Figura 3) - para gerar música mais longa ou mesmo uma música completa.


<img src="estrutura.png"> 

Embora as RNNs possam ser mais poderosas na modelagem dos padrões seqüenciais na música, mostramos que podemos melhorar a conexão das melodias nas barras adjacentes ao condicionar um modelo baseado em CNN com condições 2D. Para o trabalho futuro, podemos aprender informações seqüenciais de longo prazo, considerando as melodias de várias barras anteriores, ou usando as chamadas circunvoluções causais dilatadas como proposto no modelo WaveNet [18] em nossa CNN reflexiva.

Seguindo as idéias de reforço da aprendizagem [19], também estaremos interessados em integrar a MidiNet mais modelos computacionais que lidam com diferentes aspectos da informação de música, como o reconhecimento automático de acordes [20], a marcação automática de música [14] e a emoção musical Reconhecimento [21]. Tais modelos computacionais podem fornecer feedbacks adicionais e significativos ao modelo do gerador. Desta forma, podemos controlar mais facilmente o tipo e o sentimento da música gerada, o que pode criar novas aplicações de geração de música.

### References

[1] I. J. Goodfellow, J. Pouget-Abadie, M. Mirza, B. Xu, D. Warde-Farley, S. Ozair, A. Courville,
and Y. Bengio, “Generative adversarial nets,” in Advances in neural information processing
systems, 2014, pp. 2672–2680.

[2] I. J. Goodfellow, “NIPS 2016 tutorial: Generative adversarial networks,” arXiv preprint
arXiv:1701.00160, 2017.

[3] A. Radford, L. Metz, and S. Chintala, “Unsupervised representation learning with deep convolutional
generative adversarial networks,” arXiv preprint arXiv:1511.06434, 2015.

[4] J. Gauthier, “Conditional generative adversarial nets for convolutional face generation,” Class
Project for Stanford CS231N: Convolutional Neural Networks for Visual Recognition, Winter
semester, vol. 2014, pp. 5, 2014.

[5] T. Salimans, I. J. Goodfellow, W. Zaremba, V. Cheung, A. Radford, and X. Chen, “Improved
techniques for training GANs,” in Advances in Neural Information Processing Systems, 2016,
pp. 2226–2234.

[6] O. Mogren, “C-RNN-GAN: Continuous recurrent neural networks with adversarial training,”
arXiv preprint arXiv:1611.09904, 2016.

[7] G. Hadjeres and F. Pachet, “DeepBach: a steerable model for bach chorales generation,” arXiv
preprint arXiv:1612.01010, 2016.

[8] A. Roberts, J. Engel, C. Hawthorne, I. Simon, E. Waite, S. Oore, N. Jaques, C. Resnick, and
D. Eck, “Interactive musical improvisation with Magenta,” Neural Information Processing
Systems (NIPS), 2016.

[9] D. Eck and J. Schmidhuber, “Finding temporal structure in music: Blues improvisation with
lstm recurrent networks,” in Neural Networks for Signal Processing, 2002. Proceedings of the
2002 12th IEEE Workshop on. IEEE, 2002, pp. 747–756.

[10] N. Boulanger-Lewandowski, Y. Bengio, and P. Vincent, “Modeling temporal dependencies in
high-dimensional sequences: Application to polyphonic music generation and transcription,”
arXiv preprint arXiv:1206.6392, 2012.

[11] A. van den Oord, N. Kalchbrenner, L. Espeholt, O. Vinyals, A. Graves, et al., “Conditional
image generation with pixelCNN decoders,” in Advances in Neural Information Processing
Systems, 2016, pp. 4790–4798.

[12] M. Mirza and S. Osindero, “Conditional generative adversarial nets,” arXiv preprint
arXiv:1411.1784, 2014.

[13] A. Krizhevsky, I. Sutskever, and G. E. Hinton, “Imagenet classification with deep convolutional
neural networks,” in Advances in neural information processing systems, 2012, pp. 1097–1105.

[14] J.-Y. Liu and Y.-H. Yang, “Event localization in music auto-tagging,” in Proceedings of ACM
Multimedia, 2016, pp. 1048–1057.

[15] A. Van den Oord, S. Dieleman, and B. Schrauwen, “Deep content-based music recommendation,”
in Advances in neural information processing systems, 2013, pp. 2643–2651.

[16] X. Chen, Y. Duan, R. Houthooft, J. Schulman, I. Sutskever, and P. Abbeel, “InfoGAN: Interpretable
representation learning by information maximizing generative adversarial nets,” in
Advances in Neural Information Processing Systems, 2016, pp. 2172–2180.

[17] M. Arjovsky, S. Chintala, and L. Bottou, “Wasserstein GAN,” arXiv preprint
arXiv:1701.07875, 2017.

[18] A. van den Oord, S. Dieleman, H. Zen, K. Simonyan, O. Vinyals, A. Graves, N. Kalchbrenner,
A. Senior, and K. Kavukcuoglu, “WaveNet: A generative model for raw audio,” CoRR
abs/1609.03499, 2016.

[19] R. S. Sutton and A. G. Barto, Reinforcement learning: An introduction, vol. 1, MIT press
Cambridge, 1998.

[20] E. J. Humphrey and J. P. Bello, “Rethinking automatic chord recognition with convolutional
neural networks,” in Proc. International Conference on Machine Learning and Applications,
2012, vol. 2, pp. 357–362.

[21] Y.-H. Yang and H. H. Chen, Music Emotion Recognition, CRC Press, 2011.